# 맞춤법 교정을 통한 데이터 증강 (DA-SC: Data Augmentation with Spelling Correction)


*   문법 교정을 통해 기존 문장의 의미와 유사하면서도 형태의 변화를 주어 데이터 증강

*   네이버 맞춤법 검사기를 활용하는 Python의 한글 문법 교정 라이브러리
py-hanspell (Han, 2015)을 이용하여 텍스트 교정을 진행함


In [ ]:
# HuggingFace transformers 설치 및 NSMC 데이터셋 다운로드
!pip install transformers
!pip install git+https://github.com/ssut/py-hanspell.git

!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
!git clone https://github.com/kocohub/korean-hate-speech
!git clone https://github.com/songys/Toxic_comment_data


import pandas as pd
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW
from transformers import ElectraForMaskedLM
import random
import numpy as np
from hanspell import spell_checker

from tqdm.notebook import tqdm
from tqdm import tqdm

# GPU 사용
device = torch.device("cuda")

## NSMC Dataset

In [ ]:
train = pd.read_csv("ratings_train.txt", sep='\t').dropna(axis=0)
test = pd.read_csv("ratings_test.txt", sep='\t').dropna(axis=0)
train.head(20)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0
6,7797314,원작의 긴장감을 제대로 살려내지못했다.,0
7,9443947,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...,0
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1
9,5912145,왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?,1


In [ ]:
# 맞춤법 교정 결과 확인
k=dataset.document.iloc[:20].map(lambda x : spell_checker.check(x.replace('&', ' '))[2])
pd.DataFrame(k)

,document
0,아 더빙.. 진짜 짜증 나네요 목소리
1,흠... 포스터 보고 초등학생영화 줄.... 오버 연기조차 가볍지 않구나
2,너무재밓었다그래서보는것을추천한다
3,교도소 이야기구먼 .. 솔직히 재미는 없다.. 평점 조정
4,사이먼 페그의 익살스러운 연기가 돋보였던 영화! 스파이더맨에서 늙어 보이기만 했던 ...
5,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살 용 영화.ㅋㅋㅋ... 별 반 개도 ...
6,원작의 긴장감을 제대로 살려내지 못했다.
7,별 반 개도 아깝다 욕 나온다 이응경 길용우 연기생활이 몇 년인지.. 정말 발로해도...
8,액션이 없는데도 재미있는 몇 안 되는 영화
9,왜 이렇게 평점이 낮은 건데? 꽤 볼만한데.. 할리우드식 화려함에만 너무 길들여져 있나?


In [ ]:
# 전체 학습데이터 맞춤법교정
for i in range(14):
    print(i*10000, min((i+1)*10000, 149995))
    k=dataset.document.iloc[i*10000:min((i+1)*10000, 149995)].map(lambda x : spell_checker.check(x.replace('&', ' '))[2])
    pd.DataFrame(k).to_csv("/content/drive/MyDrive/논문/spell_check_nsmc/train_" + str(i)+".csv")

## BEEP Dataset

In [ ]:
train = pd.read_csv("/content/korean-hate-speech/labeled/train.tsv", sep = '\t')
dev = pd.read_csv("/content/korean-hate-speech/labeled/dev.tsv", sep = '\t')
print(train.shape, dev.shape)
train.head()

(7896, 4) (471, 4)


,comments,contain_gender_bias,bias,hate
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,False,others,hate
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,False,none,none
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",False,none,hate
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",False,none,none
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,True,gender,hate


In [ ]:
full_data= pd.concat([train, dev], axis =0)
full_data['id'] = np.arange(0, full_data.shape[0])
full_data.head()

,comments,contain_gender_bias,bias,hate,id
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,False,others,hate,0
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,False,none,none,1
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",False,none,hate,2
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",False,none,none,3
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,True,gender,hate,4


In [ ]:
train1 = full_data[:7896]
dev1 = full_data[7896:]

In [ ]:
# 전체 학습데이터 맞춤법교정
train2 = train1.copy()
train2['comments1'] = ''
for i in tqdm(range(7896)):
    k=spell_checker.check(train2.comments[i].replace('&', ' '))[2]
    train2.loc[i,'comments1'] = k
    if (i!=0) & (i % 1000 == 0):
        train2.to_csv("/content/drive/MyDrive/논문/spell_check_beep/train_" + str(i)+".csv")

100%|██████████| 7896/7896 [19:44<00:00,  6.67it/s]


In [ ]:
train2.to_csv("/content/drive/MyDrive/논문/spell_check_beep/train" +".csv", index = False)
dev1.to_csv("/content/drive/MyDrive/논문/spell_check_beep/dev" +".csv", index = False)

In [ ]:
# 맞춤법 교정 결과 확인
train2

,comments,contain_gender_bias,bias,hate,id,comments1
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,False,others,hate,0,(현재 호텔 주인 심정) 아 18 난 마른하늘에 날벼락 맞고 호텔 망하게 생겼는데 ...
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,False,none,none,1,....한국적인 미인의 대표적인 분... 너무나 곱고 아름다운 모습... 그 모습 ...
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",False,none,hate,2,"...못된 놈들... 남의 고통을 즐겼던 놈들.. 이젠 마땅한 처벌을 받아야지..,..."
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",False,none,none,3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데"
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,True,gender,hate,4,1. 사람 얼굴 손톱으로 긁은 것은 인격 살해이고 2. 동영상이 몰카냐? 에 걸리 ...
...,...,...,...,...,...,...
7891,힘내세요~ 응원합니다!!,False,none,none,7891,힘내세요~ 응원합니다!!
7892,힘내세요~~삼가 고인의 명복을 빕니다..,False,none,none,7892,힘내세요~~삼가 고인의 명복을 빕니다..
7893,힘내세용 ^^ 항상 응원합니닷 ^^ !,False,none,none,7893,힘내세요 ^^ 항상 응원합니다 ^^ !
7894,힘내소...연기로 답해요.나도 53살 인데 이런일 저런일 다 있더라구요.인격을 믿습...,False,none,none,7894,힘내소... 연기로 답해요. 나도 53살인데 이런 일 저런 일 다 있더라고요. 인격...
